# Setup

## Project Overview

This notebook analyzes the Yelp Academic Dataset to study restaurant review and rating patterns
across cuisines (e.g., Chinese, Korean, Italian, French, American) and U.S. states.

The analysis includes:
- Feature engineering based on restaurant categories
- Comparisons of review volume and ratings across cuisines
- Geographic analysis at the state level
- Business-focused insights on cuisine popularity and regional patterns

## Data Notice

This project uses the Yelp Academic Dataset.

Due to data size and licensing restrictions, raw dataset files
(e.g., `yelp_academic_dataset_business.json`) are **not included** in this repository.

The notebook demonstrates the full analysis workflow and logic.
Results and figures were generated locally.


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime

import duckdb


# EDA with a focus on Business

##### The relationship between review count and rating

In [ ]:
# import business
path_business = "/Users/ruihuang/Downloads/Yelp JSON/yelp_dataset/yelp_academic_dataset_business.json"

path_business = r"C:\Users\qiuwk\Desktop\Yelp Project\yelp_academic_dataset_business.json"

raw_business = pd.read_json(path_business, lines = True)
raw_business.head()

In [ ]:
# Check Geography
df = raw_business.copy()

summer_peak_large = ['CA']
summer_peak_small = ['FL']
winter_peak_large = ['DE']
winter_peak_small = ['PA']

df = df[df['state'].isin(summer_peak_large + winter_peak_large + summer_peak_small + winter_peak_small)]

def categorize_state(state, summer_peak_large, winter_peak_large, summer_peak_small, winter_peak_small):
    if state in summer_peak_large:
        return 'summer_peak_large'
    if state in winter_peak_large:
        return 'winter_peak_large'
    if state in summer_peak_small:
        return 'summer_peak_small'
    if state in winter_peak_small:
        return 'winter_peak_small'
    return 'others'

df['state_type'] = df['state'].apply(lambda x: categorize_state(x, summer_peak_large, winter_peak_large, summer_peak_small, winter_peak_small))

plt.figure(figsize=(8, 4))
sns.countplot(data=df, x='state_type', order=['summer_peak_large', 'winter_peak_large', 'summer_peak_small', 'winter_peak_small'])
plt.title("Distribution of Business by State Type")
plt.xlabel("State Type")
plt.ylabel("Count")
plt.tight_layout()
plt.show()

In [ ]:
df[['state','business_id']].groupby('state').count()

In [ ]:
# compare high vs. low rating by cusine type

df = raw_business.copy()

cuisine_map = {
        'chinese': ['Chinese', 'Szechuan', 'Cantonese', 'Dim Sum'],
        'korean': ['Korean', 'BBQ', 'KBBQ'],
        'Italian': ['Italian'],
        'French' : ['French'],
        'American' : ['American']
    }
    
for cuisine, keywords in cuisine_map.items():
    df[cuisine] = df['categories'].str.contains('|'.join(keywords), case=False, na=False)

df['rating_group'] = np.where(df['stars'] >= 4, 'high', 'low')

plot_data = df.groupby(['rating_group'])[['chinese', 'korean', 'Italian', 'French', 'American']].sum().T

plot_data.plot(kind='bar', stacked=True, figsize=(10, 5))
plt.title("High vs Low Ratings by Cuisine Type")
plt.xlabel("Cuisine Type")
plt.ylabel("Number of Businesses")
plt.xticks(rotation=45)
plt.show()


In [ ]:
df_cuisine_reviews = df.copy()


review_counts_by_cuisine = {}
review_median_by_cuisine = {}
review_mean_by_cuisine = {}


for cuisine in cuisine_map.keys():
    total_reviews = df_cuisine_reviews.loc[df_cuisine_reviews[cuisine], 'review_count'].sum()
    review_counts_by_cuisine[cuisine] = total_reviews
    
    median_reviews = df_cuisine_reviews.loc[df_cuisine_reviews[cuisine], 'review_count'].median()
    review_median_by_cuisine[cuisine] = median_reviews
    
    mean_reviews = df_cuisine_reviews.loc[df_cuisine_reviews[cuisine], 'review_count'].mean()
    review_mean_by_cuisine[cuisine] = mean_reviews


review_df = pd.DataFrame(list(review_counts_by_cuisine.items()), columns=['Cuisine', 'Total_Reviews'])
total_df = review_df.sort_values(by='Total_Reviews', ascending=False)

review_df = pd.DataFrame(list(review_median_by_cuisine.items()), columns=['Cuisine', 'Median_Reviews'])
median_df = review_df.sort_values(by='Median_Reviews', ascending=False)

review_df = pd.DataFrame(list(review_mean_by_cuisine.items()), columns=['Cuisine', 'Mean_Reviews'])
mean_df = review_df.sort_values(by='Mean_Reviews', ascending=False)

In [ ]:
total_df

In [ ]:
median_df

In [ ]:
mean_df

In [ ]:
top_states_by_cuisine = {}

for cuisine in cuisine_map.keys():
   
    cuisine_df = df[df[cuisine]]
    
    state_counts = cuisine_df['state'].value_counts()
    
    top_states = state_counts.head(3)
    
    top_states_by_cuisine[cuisine] = top_states


for cuisine, top_states in top_states_by_cuisine.items():
    print(f"\nTop 3 states for {cuisine.capitalize()} cuisine:")
    print(top_states)

In [ ]:
result = []

for cuisine in cuisine_map.keys():
 
    cuisine_df = df[df[cuisine]]

   
    pa_df = cuisine_df[cuisine_df['state'] == 'PA']
    non_pa_df = cuisine_df[cuisine_df['state'] != 'PA']

    
    pa_avg_reviews = pa_df['review_count'].mean()
    pa_avg_stars = pa_df['stars'].mean()

    
    non_pa_avg_reviews = non_pa_df['review_count'].mean()
    non_pa_avg_stars = non_pa_df['stars'].mean()

   
    result.append({
        'Cuisine': cuisine.capitalize(),
        'PA_Avg_Reviews': pa_avg_reviews,
        'PA_Avg_Stars': round(pa_avg_stars, 2),
        'Other_Avg_Reviews': round(non_pa_avg_reviews, 2),
        'Other_Avg_Stars': round(non_pa_avg_stars, 2)
    })


review_comparison_df = pd.DataFrame(result)
review_comparison_df


In [ ]:
# this block/cell is to...
def analyze_review_rating_relationship(df):
    import scipy.stats as stats
    result = df.groupby('rating_group')['review_count'].agg(['mean', 'median', 'count'])
    stat, p_value = stats.mannwhitneyu(
        df[df['rating_group'] == 'high']['review_count'],
        df[df['rating_group'] == 'low']['review_count']
    )

    return result, p_value


result, p_value = analyze_review_rating_relationship(df)
print(p_value)
result

# Analysis on Ratings

##### The rating trend of elite users

In [ ]:
import pandas as pd
import duckdb

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


path_user = r'/Users/ruihuang/Downloads/Yelp JSON/yelp_dataset/yelp_academic_dataset_user.json'
path_review = r'/Users/ruihuang/Downloads/Yelp JSON/yelp_dataset/yelp_academic_dataset_review.json'
path_business = r'/Users/ruihuang/Downloads/Yelp JSON/yelp_dataset/yelp_academic_dataset_business.json'

reviews = duckdb.read_json(path_review)
business = duckdb.read_json(path_business)
user = duckdb.read_json(path_user)

In [ ]:
# checking elite users

query = '''
SELECT avg(case when elite like '%2013%' then average_stars end) as avg_rating_2013,
avg(case when elite like '%2014%' then average_stars end) as avg_rating_2014,
avg(case when elite like '%2015%' then average_stars end) as avg_rating_2015,
avg(case when elite like '%2016%' then average_stars end) as avg_rating_2016,
avg(case when elite like '%2017%' then average_stars end) as avg_rating_2017,

avg(case when elite like '%2013%' then review_count end) as avg_review_count_2013,
avg(case when elite like '%2014%' then review_count end) as avg_review_count_2014,
avg(case when elite like '%2015%' then review_count end) as avg_review_count_2015,
avg(case when elite like '%2016%' then review_count end) as avg_review_count_2016,
avg(case when elite like '%2017%' then review_count end) as avg_review_count_2017

from user
'''

elite_user_stats = duckdb.sql(query).df()

temp_dict = {'year':[2013, 2014, 2015, 2016, 2017],
             'avg_rating':[elite_user_stats['avg_rating_2013'].iloc[0],
                         elite_user_stats['avg_rating_2014'].iloc[0],
                         elite_user_stats['avg_rating_2015'].iloc[0],
                         elite_user_stats['avg_rating_2016'].iloc[0],
                         elite_user_stats['avg_rating_2017'].iloc[0]],
             'avg_review_count':[elite_user_stats['avg_review_count_2013'].iloc[0],
                               elite_user_stats['avg_review_count_2014'].iloc[0],
                               elite_user_stats['avg_review_count_2015'].iloc[0],
                               elite_user_stats['avg_review_count_2016'].iloc[0],
                               elite_user_stats['avg_review_count_2017'].iloc[0]]}

elite_user_stats = pd.DataFrame(temp_dict)

elite_user_stats['yoy_rating_change'] = elite_user_stats['avg_rating'] - elite_user_stats['avg_rating'].shift(1)

elite_user_stats

##### Rating trend of all users

In [ ]:
query = '''
SELECT 
    avg(average_stars) as avg_rating_all_users,
    avg(review_count) as avg_review_count_all_users,
    
   
    avg(case when yelping_since between '2013-01-01' and '2013-12-31' then average_stars end) as avg_rating_2013,
    avg(case when yelping_since between '2014-01-01' and '2015-12-31' then average_stars end) as avg_rating_2014,
    avg(case when yelping_since between '2015-01-01' and '2015-12-31' then average_stars end) as avg_rating_2015,
    avg(case when yelping_since between '2016-01-01' and '2016-12-31' then average_stars end) as avg_rating_2016,
    avg(case when yelping_since between '2017-01-01' and '2017-12-31' then average_stars end) as avg_rating_2017,
    
 
    avg(case when yelping_since between '2013-01-01' and '2013-12-31' then review_count end) as avg_review_count_2013,
    avg(case when yelping_since between '2014-01-01' and '2014-12-31' then review_count end) as avg_review_count_2014,
    avg(case when yelping_since between '2015-01-01' and '2015-12-31' then review_count end) as avg_review_count_2015,
    avg(case when yelping_since between '2016-01-01' and '2016-12-31' then review_count end) as avg_review_count_2016,
    avg(case when yelping_since between '2017-01-01' and '2017-12-31' then review_count end) as avg_review_count_2017,
    
   
    count(case when yelping_since between '2013-01-01' and '2013-12-31' then user_id end) as user_count_2013,
    count(case when yelping_since between '2013-01-01' and '2014-12-31' then user_id end) as user_count_2014,
    count(case when yelping_since between '2013-01-01' and '2015-12-31' then user_id end) as user_count_2015,
    count(case when yelping_since between '2013-01-01' and '2016-12-31' then user_id end) as user_count_2016,
    count(case when yelping_since between '2013-01-01' and '2017-12-31' then user_id end) as user_count_2017
    
FROM user
WHERE yelping_since between '2013-01-01' and '2017-12-31' 
'''

all_user_stats = duckdb.sql(query).df()

temp_dict = {'year':[2013, 2014, 2015, 2016, 2017],
             'avg_rating':[all_user_stats['avg_rating_2013'].iloc[0],
                         all_user_stats['avg_rating_2014'].iloc[0],
                         all_user_stats['avg_rating_2015'].iloc[0],
                         all_user_stats['avg_rating_2016'].iloc[0],
                         all_user_stats['avg_rating_2017'].iloc[0]],
             'avg_review_count':[all_user_stats['avg_review_count_2013'].iloc[0],
                               all_user_stats['avg_review_count_2014'].iloc[0],
                               all_user_stats['avg_review_count_2015'].iloc[0],
                               all_user_stats['avg_review_count_2016'].iloc[0],
                               all_user_stats['avg_review_count_2017'].iloc[0]],
             'user_count':[all_user_stats['user_count_2013'].iloc[0],
                         all_user_stats['user_count_2014'].iloc[0],
                         all_user_stats['user_count_2015'].iloc[0],
                         all_user_stats['user_count_2016'].iloc[0],
                         all_user_stats['user_count_2017'].iloc[0]]}

all_users_stats = pd.DataFrame(temp_dict)

all_users_stats['yoy_rating_change'] = all_users_stats['avg_rating'] - all_users_stats['avg_rating'].shift(1)

all_users_stats

##### Top 20 Restaurants

In [ ]:
# Get the top 20 restaurants

query = """
WITH restaurant_stats AS (
    SELECT business_id,
        COUNT(*) AS review_count,
        AVG(stars) AS avg_rating
    FROM reviews 
    WHERE date between '2013-01-01' and '2017-12-31'
    GROUP BY business_id
    ORDER BY review_count DESC
    LIMIT 20
)


SELECT 
    r.business_id,
    b.name,
    b.city,
    b.state,
    b.categories,
    r.review_count,
    r.avg_rating,
    b.stars AS business_avg_rating,
    b.review_count AS business_total_reviews
FROM restaurant_stats r
JOIN business b ON r.business_id = b.business_id
ORDER BY r.review_count DESC
"""


top_restaurants = duckdb.sql(query).df()


top_restaurants.insert(0, 'rank', range(1, len(top_restaurants)+1))

top_restaurants

In [ ]:
import pandas as pd
import duckdb
import matplotlib.pyplot as plt


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


path_review = r'/Users/ruihuang/Downloads/Yelp JSON/yelp_dataset/yelp_academic_dataset_review.json'
path_business = r'/Users/ruihuang/Downloads/Yelp JSON/yelp_dataset/yelp_academic_dataset_business.json'

review = duckdb.read_json(path_review)
business = duckdb.read_json(path_business)


query_top20 = """
WITH base AS (
    SELECT DATE_PART('year',date) as year,
    business_id,
    COUNT(review_id) as review_count,
    AVG(stars) as avg_rating
    FROM review
    WHERE date BETWEEN '2013-01-01' AND '2017-12-31'
    AND business_id IN ('_ab50qdWOk0DdB6XOrBitw', 'ytynqOUb3hjKeJfRj5Tshw', 'GXFMD0Z4jEVZBCsbPf4CTQ', 'ac1AeYqs8Z4_e2X5M3if2A',
    'VQcCL9PiNL_wkGf-uF3fjg', 'GBTPC53ZrG1ZBY3DT8Mbcw', '_C7QiQQc47AOEv4PE3Kong', 'iSRTaT9WngzB8JJ2YKJUig', 
    '6a4gLLFSgr-Q6CZXDLzBGQ', 'yPSejq3_erxo9zdVYTBnZA', 'oBNrLz4EDhiscSlbOl8uAw', 'gTC8IQ_i8zXytWSly3Ttvg',
    'I_3LMZ_1m2mzR0oLIOePIg', 'PP3BBaVxZLcJU54uP_wL6Q', '1b5mnK8bMnnju_cvU65GqQ', 'VaO-VW3e1kARkU9bP1E7Fw', 
    'QHWYlmVbLC3K6eglWoHVvA', 'qb28j-FNX1_6xm7u372TZA', 'VAeEXLbEcI9Emt9KGYq9aA', '9PZxjhTIU7OgPIzuGi89Ew')
    Group by 1,2
)

SELECT 
    base.*,
    b.name,
    b.city,
    b.state,
    b.categories,
    LAG(base.avg_rating, 1, 0) OVER (PARTITION BY base.business_id ORDER BY base.year ASC) as previous_year_avg_rating,
    CASE WHEN previous_year_avg_rating THEN null 
         ELSE base.avg_rating / previous_year_avg_rating -1 
         END as yoy_rating_change
FROM base JOIN business b ON base.business_id = b.business_id
"""

top20_restaurants = duckdb.sql(query_top20).df()

top20_restaurants

In [ ]:
all_user_temp = all_users_stats[['year','yoy_rating_change']].copy()
all_user_temp = all_user_temp.rename(columns={'yoy_rating_change':'yoy_rating_change_all'})

elite_user_temp = elite_user_stats[['year','yoy_rating_change']].copy()
elite_user_temp = elite_user_temp.rename(columns={'yoy_rating_change':'yoy_rating_change_elite'})

final_table = top20_restaurants.merge(all_user_temp, on='year', how='left').merge(elite_user_temp, on='year', how='left')

In [ ]:
final_table